In [1]:
# Assuming we launched from goliath/notebooks, add the codebase itself to the path
import sys
sys.path.insert(0, '../')

In [2]:
%cd ..
%bookmark GOLIATH_HOME

/mnt/home/julietamartinez/goliath


In [3]:
# Set your data path here!
data_path = "/mnt/captures/oss/goliath/s--20230306--1509--AXE977--pilot--ProjectGoliath--ClothedBody/"

In [4]:
# !mkdir -p checkpoints/clothed-body/AXE977
# !wget https://fb-baas-f32eacb9-8abb-11eb-b2b8-4857dd089e15.s3.amazonaws.com/goliath-for-review-full/checkpoints/AXE977/latest.pt -O checkpoints/clothed-body/AXE977/latest.pt

In [ ]:
# !mkdir -p checkpoints/clothed-body/AXE977
# !wget https://fb-baas-f32eacb9-8abb-11eb-b2b8-4857dd089e15.s3.amazonaws.com/goliath-for-review-full/checkpoints/AXE977/config.yml -O checkpoints/clothed-body/AXE977/config.yml

In [18]:
%cd GOLIATH_HOME

# Load a pre-built config, it will make things easier
from omegaconf import DictConfig, OmegaConf
from ca_code.utils.dataloader import BodyDataset, collate_fn
from torch.utils.data import DataLoader

# config = OmegaConf.load("checkpoints/clothed-body/AXE977/config.yml")
config = OmegaConf.load("config/mesh_vae_example.yml")

# Create the dataset that we will use
config.data.root_path = data_path
config.data.shared_assets_path = "data/shared/static_assets_body.pt"
config.data.split = "test"
config.data.fully_lit_only = True
config.data.partially_lit_only = False
# config.data.frames_subset = config.test.data.frames_subset

dataset = BodyDataset(**config.data)

config.dataloader.shuffle = False
config.dataloader.batch_size = 1
config.dataloader.num_workers = 4

# dataset.cameras = ["400933"]
loader = DataLoader(dataset, collate_fn=collate_fn, **config.dataloader)

[2024-12-03 16:27:15][INFO][ca_code.utils.dataloader]:Found 212 cameras in the calibration file
[2024-12-03 16:27:15][INFO][ca_code.utils.dataloader]:Left with 212 cameras after filtering for zips present in image/ folder


(bookmark:GOLIATH_HOME) -> /mnt/home/julietamartinez/goliath
/mnt/home/julietamartinez/goliath


In [20]:
# Create model, load weights
from addict import Dict as AttrDict
from ca_code.utils.module_loader import load_from_config
from ca_code.utils.train import load_checkpoint, to_device

ckpt_path = "checkpoints/clothed-body/AXE977/latest.pt"

static_assets = AttrDict(dataset.static_assets)

device = "cuda"
model = (
        load_from_config(
            config.model,
            assets=static_assets,
        )
        .to(device)
        .eval()
    )

model.learn_blur_enabled = False
model.cal_enabled = False

# loading model checkpoint
load_checkpoint(ckpt_path, modules={"model": model}, strict=False)

tensor([[   682,   7356,    682,    682],
        [   692,   7360,    692,    692],
        [   596,   7359,    596,    596],
        ...,
        [148147, 148147, 148147, 148147],
        [148021, 148021, 148021, 148021],
        [148028, 148028, 148028, 148028]], dtype=torch.int32)


[2024-12-03 16:28:10][INFO][ca_code.utils.geom]:impainting index image might take a while for sizes >= 1024
[2024-12-03 16:28:12][INFO][ca_code.models.mesh_vae]:ConvDecoder: n_channels = [64, 32, 16, 8, 4]
/mnt/home/julietamartinez/goliath/ca_code/nn/layers.py:233: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  wn = WeightNorm.apply(
[2024-12-03 16:28:12][WARNING][ca_code.nn.color_cal]:Requested color-calibration identity camera not present, defaulting to 400870.
[2024-12-03 16:28:12][INFO][ca_code.utils.train]:loading checkpoint checkpoints/clothed-body/AXE977/latest.pt


RuntimeError: Error(s) in loading state_dict for AutoEncoder:
	size mismatch for pixel_cal.bias: copying a param with shape torch.Size([133, 1, 166, 256]) from checkpoint, the shape in current model is torch.Size([212, 1, 166, 256]).
	size mismatch for learn_blur.weights_raw: copying a param with shape torch.Size([133, 3]) from checkpoint, the shape in current model is torch.Size([212, 3]).
	size mismatch for cal.holder.params: copying a param with shape torch.Size([133, 6]) from checkpoint, the shape in current model is torch.Size([212, 6]).

In [7]:
from torchvision.utils import make_grid, save_image
from ca_code.utils.image import linear2srgb
import torch as th

batch_filter_fn = dataset.batch_filter

# forward
for i, batch in enumerate(loader):
    print(f"{i}/{len(loader)}")
    batch = to_device(batch, device)
    batch_filter_fn(batch)

    with th.no_grad():
        preds = model_p(**batch, index=[180 + i])

    # visualizing
    rgb_preds_grid = make_grid(linear2srgb(preds["rgb"]), nrow=4)
    save_image(rgb_preds_grid, f"preds/{i}.png")

data/shared/static_assets_head.pt*


In [9]:
ls data/shared

legacy/      static_assets_body.pt*       static_assets_hand_right.pt*
LICENSE.md*  static_assets_hand_left.pt*  static_assets_head.pt*
